### Import and set-up

In [5]:
from dataclasses import dataclass, field
from typing import List, Tuple, Any

In [119]:
'''
NOTES FOR EDITS:
-If add a new op that requires args[0], add to list "needs_args"
-
'''

Instruction = Tuple[str, Any]

# create boilerplate for dataclass CPU
# dataclass decorator creates class boilerplates that already have __init__, __repr__, and __eq__ defined
@dataclass
class CPU:
    mem: List[int] = field(default_factory=lambda: [0]*32)      # 32 integer cells of RAM
    acc: int = 0                                                # Accumulator
    idx: int = 0                                                # Index register
    pc: int = 0                                                 # Program counter
    halted: bool = False
    program: List[Instruction] = field(default_factory=list)    # program is a List of Instructions (list of tuples)

    # load program, with starting values for each field except mem
    def load_program(self, program: List[Instruction]):
        '''Load a list of instructions and reset CPU state.'''
        self.program = program
        self.pc = 0
        self.halted = False
        self.acc = 0
        self.idx = 0
        # memory persists until you overwrite cpu.mem explicitly

    def dump(self):
        '''Snapshot of key state for quick inspection'''
        return{
            'PC': self.pc,
            'ACC': self.acc,
            'IDX': self.idx,
            'HALTED': self.halted,
            'MEMO..7': self.mem[:8]
        }
    
    def step(self):
        '''Execute exactly the next instruction'''
        # if halted or pc is not between 0 and len(program), set halted = True
        if self.halted or not (0 <= self.pc <= len(self.program)):
            self.halted = True 
            return
        
        # set-up
        instr = self.program[self.pc]
        # unpack instruction into op and *args or 
        op, *args = instr if isinstance(instr, tuple) else (instr,)

        # reset variables
        target_index = None
        addr = None

        
        # enforce that *args[0] is int
        if len(args) >= 1 and not isinstance(args[0], int):
            raise TypeError(f'({op}, {args}): second tuple item must be an int')
        
        # enforce that certain ops have args[0]
        # note: 
        needs_args = {
            'LOADI',
            'LOAD_ABS',
            'LOAD_IDX_OFF',
            'STORE_ABS',
            'STORE_IDX_OFF',
            'ADDI',
            'ADD_ABS',
            'JUMP_ABS',
            'JUMP_REL'
            }
        if op in needs_args:
            if not args:
                raise ValueError(f'op {op} must have a int.')

        
        # HELPER FUNCTIONS
        # use with any command that operates on an ABS memory address
        def _get_address(self, args: any) -> int:
            addr = args[0]
            if not (0 <= addr <= len(self.mem)):
                raise IndexError(f'({op}, {args}) in {self.pc} contains a bad memory address.')
            return addr

        # use with any command that operates on the IDX
        def _get_target_idx(self, op: str, args: any) -> int:
            if 'OFF' in op: 
                off = args[0]
                target_idx = self.idx + off
            elif 'DEC' in op:
                target_idx = self.idx - 1
            elif 'INC' in op:
                target_idx = self.idx + 1

            if not 0 <= target_idx < len(self.mem):
                raise IndexError(f'({op}, {args[0] if args else ''}) in {self.pc} results in IDX out of range.')
            
            return target_idx 

        # use with any command that operates on the memory slot @IDX
        def _check_idx(self):
            if not (0 <= self.idx < len(self.mem)):
                raise IndexError(f'IDX out of range: {self.idx} in step {self.pc}')
            
        # use with any command that operates on self.pc
        def _check_pc(self, args: any) -> int:
            target_step = self.pc + args[0]
            if not (0 <= target_step < len(self.program)):
                raise IndexError(f'Bad pc: {target_step}')
            return target_step


        if op == 'LOADI':           # ACC <- immediate
            self.acc = args[0]
        elif op == 'LOAD_ABS':      # ACC <- mem[addr]
            addr = _get_address(instr, args)
            self.acc = self.mem[addr]
        elif op == 'LOAD_IDX':      # ACC <- mem[idx]
            _check_idx()
            self.acc = self.mem[self.idx]
        elif op == 'LOAD_IDX_OFF':  # ACC <- mem[idx+off]
            target_index = _get_target_idx(instr, args)
            self.ACC = self.mem[target_index]
        elif op == 'STORE_ABS':     # mem[addr] <- ACC
            addr = _get_address(instr, args)
            self.mem[addr] = self.acc
        elif op == 'STORE_IDX':     # mem[idx] <- ACC
            _check_idx()
            self.mem[self.idx] = self.acc
        elif op == 'STORE_IDX_OFF':  # mem[idx+off] <- ACC
            target_index = _get_target_idx(instr, args)
            self.mem[target_index] = self.acc
        elif op == 'ADDI':          # acc = acc + immediate
            self.acc = self.acc + args[0]
        elif op == 'ADD_ABS':       # acc = acc + mem[addr]
            addr = _get_address(instr, args)
            self.acc = self.acc + self.mem[addr]
        elif op == 'ADD_IDX':       # acc = acc + mem[idx]
            _check_idx()
            self.acc += self.mem[self.idx]
        elif op == 'INC_IDX':
            target_index = _get_target_idx(instr, args)     
            self.idx = target_index
        elif op == 'DEC_IDX':
            target_index = _get_target_idx(instr, args)     
            self.idx = target_index
        elif op == 'JUMP_ABS':      # Absolute jump
            target_step = int(args[0])
            self.pc = target_step
        elif op == 'JUMP_REL':      # Relative jump
            target_step = _check_pc()
            self.pc = target_step
        elif op == 'HALT':          # stop execution
            self.halted = True
        else:
            raise NotImplementedError(f'Unknown op: {op}')
        
        self.pc += 1    # question, what are implications of putting this line before vs after action

    def run(self, max_steps=100):
        '''Run up to max_steps or until halted.'''
        steps = 0
        while not self.halted and steps < max_steps:
            self.step()
            steps += 1
        return steps


In [120]:
prog = [
    ('LOADI', -7),
    ('STORE_ABS', 0),
    ('LOADI', 12),
    ('STORE_ABS', 1),
    ('LOAD_ABS', 1),
    ('HALT',),
]

In [121]:
cpu = CPU()
cpu.load_program(prog)
cpu.run()
print(cpu.dump())


AttributeError: 'tuple' object has no attribute 'mem'

In [41]:
args = [3, 4, 'bobi']

all([isinstance(arg, int) for arg in args])

False